In [1]:
from ipywidgets import interactive, widgets, fixed
from surfer import Brain as surface
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import minmax_scale, scale

import os
import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as random

# spectrome imports
from spectrome.brain import Brain
from spectrome.utils import functions, path
import csv


In [2]:

import random as random

In [3]:
def eigmode2plot(labels, slider_alpha, slider_k, em, lap_type = 'complex'):
    ## spectrome brain object with HCP connectome:
    hcp_dir = '../data'
    brain = Brain.Brain()
    brain.add_connectome(hcp_dir)
    brain.reorder_connectome(brain.connectome, brain.distance_matrix)
    brain.bi_symmetric_c()
    brain.reduce_extreme_dir()
    ## compute eigenmodes:
    if lap_type == 'complex':
        brain.decompose_complex_laplacian(alpha=slider_alpha, k=slider_k, num_ev=86)
        lh_cort_eigs = minmax_scale((estU[0:34, em - 1]))
        rh_cort_eigs = minmax_scale((estU[34:68,em - 1]))
    elif lap_type == 'real':
        brain.add_regular_laplacian_eigenmodes(alpha = slider_alpha, num_ev = 86, vis = False)
        lh_cort_eigs = minmax_scale((estU[0:34, em - 1]))
        rh_cort_eigs = minmax_scale((estU[34:68,em - 1]))
    lh_cort_padded = np.insert(lh_cort_eigs, [0,3], [0,0])
    rh_cort_padded = np.insert(rh_cort_eigs, [0,3], [0,0])
    lh_cort = lh_cort_padded[labels]
    rh_cort = rh_cort_padded[labels]
    return lh_cort, rh_cort, brain.norm_eigenmodes, em

In [4]:
def eigmode_widget(labels, surf_brain, slider_alpha, slider_k, em, hemi):
    turbo = ListedColormap(turbo_colormap_data)
    hcp_dir = '../data'
    brain = Brain.Brain()
    brain.add_connectome(hcp_dir)
    brain.reorder_connectome(brain.connectome, brain.distance_matrix)
    brain.bi_symmetric_c()
    brain.reduce_extreme_dir()
    brain.decompose_complex_laplacian(alpha=slider_alpha, k=slider_k, num_ev=86)
    lh_cort_eigs = minmax_scale((estU[0:34, :]))[:, em-1]
    rh_cort_eigs = minmax_scale((estU[34:68, :]))[:, em-1]
    lh_cort_padded = np.insert(lh_cort_eigs, [0,3], [0,0])
    rh_cort_padded = np.insert(rh_cort_eigs, [0,3], [0,0])
    lh_cort = lh_cort_padded[labels]
    rh_cort = rh_cort_padded[labels]
    if hemi == 'Left':
        surf_brain = surface(subject_id, "lh", surf, background = "white", alpha = 0.3, title = "Eigen Modes of Complex LaPlacian", subjects_dir = SUBJECTS_DIR)
        surf_brain.add_data(lh_cort, hemi = "lh", thresh = 0, colormap = turbo, remove_existing = True)
        surf_brain.scale_data_colormap(color_fmin, color_fmid, color_fmax, transparent = False)
    elif hemi == 'Right':
        surf_brain = surface(subject_id, "rh", surf, background = "white", alpha = 0.3, title = "Eigen Modes of Complex LaPlacian", subjects_dir = SUBJECTS_DIR)
        surf_brain.add_data(rh_cort, hemi = "rh", thresh = 0, colormap = turbo, remove_existing = True)
        surf_brain.scale_data_colormap(color_fmin, color_fmid, color_fmax, transparent = False)
    elif hemi == 'Both':
        surf_brain = surface(subject_id, "both", surf, background = "white", alpha = 0.3, title = "Eigen Modes of Complex LaPlacian",  subjects_dir = SUBJECTS_DIR)
        surf_brain.add_data(lh_cort, hemi = "lh", thresh = 0, colormap = turbo, remove_existing = True)
        surf_brain.add_data(rh_cort, hemi = "rh", thresh = 0, colormap = turbo, remove_existing = False)
        surf_brain.scale_data_colormap(color_fmin, color_fmid, color_fmax, transparent = False)
    return lh_cort, rh_cort, brain.norm_eigenmodes

In [1]:

estU1 = (np.load('/Users/feijiang/Documents/dynamic/segUsub1chg.npy'))
nsim = estU1.shape[1]
turbo_colormap_data = np.load('turbocolormap.npy')
turbo = ListedColormap(turbo_colormap_data)
color_fmin, color_fmid, color_fmax = 0, 0.5, 1
## Load Yeo 2017's canonical network maps:
fc_dk = np.load('../data/com_dk.npy', allow_pickle = True).item()
fc_dk_normalized = pd.read_csv('../data/DK_dictionary_normalized.csv').set_index('Unnamed: 0')



# set up Pysurfer variables
subject_id = "fsaverage"
hemi = ["lh","rh"]
surf = "white"
hemi_side = "lh"
SUBJECTS_DIR = '/Applications/freesurfer/subjects/'
aparc_file = os.path.join(SUBJECTS_DIR, subject_id, "label", hemi_side + ".aparc.annot")
labels, ctab, names = nib.freesurfer.read_annot(aparc_file)

###
for sampix in range(nsim):
    random.seed(2020)
    #estU = estU[:, sampix]#estU.transpose()
    
    estU = np.reshape(estU1[:, sampix], (68, 1))
    estU.shape
    param1 = np.array([2.5, 2])
    param2 = np.array([0.2, 8])
    em1 = sampix
    em = 1


# #%matplotlib inline
# interactive(eigmode_widget, labels = fixed(labels), surf_brain = fixed(sb), 
#             slider_alpha = widgets.FloatSlider(min=0.2,max=5,step=0.1,value=2.5, description = 'Coupling Strength',continuous_update=False),
#             slider_k = widgets.IntSlider(min = 0, max = 50, step = 1, value = 2, description = 'Wave Number',continuous_update=False),
#             em = widgets.IntSlider(min = 1, max = 10, step = 1, value = 3, description = 'Eigenmode Number',continuous_update=False),
#             hemi = widgets.RadioButtons(options = ['Left', 'Right', 'Both'], value = 'Left', description = 'Select hemisphere'))

# ## define turbo colormap

   

    lh_cort, rh_cort, eigs, em = eigmode2plot(labels, slider_alpha = param1[0], slider_k = param1[1], em=em)
## initialize pysurfer rendering:
    sb = surface(subject_id, 'lh', surf, background = "white", alpha = 0.3, title = "Singular vectors", subjects_dir = SUBJECTS_DIR)
    sb.add_data(lh_cort, hemi = 'lh', thresh = 0, colormap = turbo, remove_existing = True)
    sb.scale_data_colormap(color_fmin, color_fmid, color_fmax, transparent = False)


## show lateral and medial views of left hemisphere and save figures
    sb.show_view('lat')
    sb.save_image('figures/%s_lat_MEG%1d.pdf' % ('sub1chg',  sampix))

    sb.show_view('med')
    sb.save_image('figures/%s_med_MEG%1d.pdf' % ('sub1chg',  sampix))

    sb = surface(subject_id, "both", surf, background = "white", alpha = 0.3, title = "Singular vectors", subjects_dir = SUBJECTS_DIR)
    sb.add_data(rh_cort, hemi = 'rh', thresh = 0, colormap = turbo, remove_existing = True)
    sb.add_data(lh_cort, hemi = 'lh', thresh = 0, colormap = turbo, remove_existing = True)
    sb.scale_data_colormap(color_fmin, color_fmid, color_fmax, transparent = False)
    sb.show_view('dor')
    sb.save_image('figures/%s_dor_MEG%1d.pdf' % ('sub1chg',  sampix))

NameError: name 'np' is not defined

In [3]:
estU1 = (np.load('/Users/feijiang/Documents/dynamic/segUsub1Im.npy'))

In [5]:
nsim

10

In [4]:
sampix

37

In [6]:
segU.shape[1]

NameError: name 'segU' is not defined

In [7]:
segU.shape


NameError: name 'segU' is not defined

In [9]:
estU.shape


(68, 1)

In [10]:
estU = (np.load('/Users/feijiang/Documents/dynamic/segU.npy'))

In [11]:
estU.shape


(68, 3)

In [15]:
nsim

3

In [22]:
estU1[:, 5] - estU1[:, 4]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [13]:
estU1.shape

(68, 1)

In [4]:
estU1 = (np.load('/Users/feijiang/Documents/dynamic/segUchg.npy'))

In [5]:
estU1.shape

(68, 12)